In [8]:
pip install opencv-python pytesseract pandas pillow

Note: you may need to restart the kernel to use updated packages.


In [9]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [10]:
import cv2
import pytesseract
from PIL import Image
import pandas as pd
import re
import os
import uuid

# Load your ingredients database
def load_ingredient_db(csv_path="ingredients_database.csv"):
    df = pd.read_csv(csv_path)
    db = {
        row["ingredient"].lower(): {
            "label": row["label"],
            "explanation": row["explanation"]
        } for _, row in df.iterrows()
    }
    return db

# OCR text extraction
def extract_text_from_image(image_path):
    image = Image.open(image_path)
    return pytesseract.image_to_string(image)

# Ingredient name extraction
def extract_ingredients(text):
    words = re.split(r'[,\n]', text.lower())
    cleaned = [re.sub(r'[^a-zA-Z ]+', '', w).strip() for w in words]
    return list(set([w for w in cleaned if len(w) > 2]))

# Ingredient analysis
def analyze_ingredients(ingredients, db):
    results = []
    for ing in ingredients:
        if ing in db:
            info = db[ing]
            results.append({
                "ingredient": ing,
                "label": info["label"],
                "explanation": info["explanation"]
            })
    return results

# Webcam capture function
def capture_image_from_webcam(output_path="captured_label.jpg"):
    cap = cv2.VideoCapture(0)
    print("📷 Press SPACE to take a photo, ESC to exit.")

    while True:
        ret, frame = cap.read()
        cv2.imshow("LabelWise Scanner - Press SPACE to Capture", frame)

        key = cv2.waitKey(1)
        if key % 256 == 27:  # ESC
            print("❌ Capture canceled.")
            break
        elif key % 256 == 32:  # SPACE
            cv2.imwrite(output_path, frame)
            print(f"✅ Image saved to {output_path}")
            break

    cap.release()
    cv2.destroyAllWindows()

# Main scanner pipeline
def scan_label_from_camera(csv_path="ingredients_database.csv"):
    image_path = f"label_{uuid.uuid4().hex}.jpg"
    capture_image_from_webcam(image_path)

    db = load_ingredient_db(csv_path)
    text = extract_text_from_image(image_path)
    ingredients = extract_ingredients(text)
    results = analyze_ingredients(ingredients, db)

    if not results:
        print("⚠️ No known ingredients detected.")
    else:
        print("\n📊 Analysis Result:")
        for r in results:
            print(f"- {r['ingredient'].title()} ({r['label']})")
            print(f"  → {r['explanation']}\n")

if __name__ == "__main__":
    scan_label_from_camera()
 

📷 Press SPACE to take a photo, ESC to exit.
✅ Image saved to label_457953a2924047879b5b7b73d4e97a2e.jpg
⚠️ No known ingredients detected.
